# Introduction

In [34]:
from sqlalchemy import create_engine, inspect, text, MetaData, select

import pandas as pd

engine = create_engine("sqlite:///data/Chinook.sqlite")
insp = inspect(engine)


In [31]:
table_name = insp.get_table_names()
print(table_name)

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


# Querying the Database

In [42]:
with engine.connect() as con:
    rs = con.execute(text("SELECT * FROM Genre"))
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()


Otra forma sin uso de sql crudo

In [41]:
metadata = MetaData()
genre = Table("Genre", metadata, autoload_with=engine)

with engine.connect() as con:
    stmt = select(genre)  # SELECT * FROM Genre
    rs = con.execute(stmt)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    

In [53]:
with engine.connect() as con:
    rs = con.execute(text("SELECT * FROM Album"))
    df = pd.DataFrame(rs.fetchmany(size = 3))
    df.columns = rs.keys()

df

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2


In [54]:
    
with engine.connect() as con:
    rs = con.execute(text("SELECT LastName, Title FROM Employee"))
    df = pd.DataFrame(rs.fetchmany(size=3))
    df.columns = rs.keys()
df

,LastName,Title
0,Adams,General Manager
1,Edwards,Sales Manager
2,Peacock,Sales Support Agent


In [55]:
with engine.connect() as con:
    rs = con.execute(text("SELECT * FROM Employee WHERE EmployeeId >= 6"))
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    
df

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
1,7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
2,8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [58]:
with engine.connect() as con:
    rs = con.execute(text("SELECT * FROM Employee ORDER BY BirthDate"))
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

df.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
3,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
4,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


# Relational Databases With Pandas

In [61]:
df = pd.read_sql("SELECT * FROM Employee", engine)

df.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [62]:

df = pd.read_sql_query("SELECT * FROM Employee WHERE EmployeeId >= 6 ORDER BY BirthDate",engine)
df

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com
1,7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
2,6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com


# Advanced querying

In [63]:
from sqlalchemy import create_engine, inspect, text
import pandas as pd

engine_n = create_engine("sqlite:///data/northwind.sqlite")
insp_n = inspect(engine_n)

print(insp_n.get_table_names())

['Categories', 'CustomerCustomerDemo', 'CustomerDemographics', 'Customers', 'EmployeeTerritories', 'Employees', 'Order Details', 'Orders', 'Products', 'Regions', 'Shippers', 'Suppliers', 'Territories']


In [65]:
df = pd.read_sql_query("""
                 SELECT OrderID, CompanyName FROM Orders
                 INNER JOIN Customers 
                 ON Orders.CustomerID = Customers.CustomerID
                 """, engine_n)

df.head()

,OrderID,CompanyName
0,10248,Vins et alcools Chevalier
1,10249,Toms Spezialitäten
2,10250,Hanari Carnes
3,10251,Victuailles en stock
4,10252,Suprêmes délices


Con Chinook.sql

In [66]:
df = pd.read_sql_query("""
    SELECT * FROM PlaylistTrack
    INNER JOIN Track 
    ON PlaylistTrack.TrackId = Track.TrackId
    WHERE Milliseconds < 250000
""",engine)
df.head()

,PlaylistId,TrackId,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,3390,3390,One and the Same,271,2,23,None,217732,3559040,0.99
1,1,3392,3392,Until We Fall,271,2,23,None,230758,3766605,0.99
2,1,3393,3393,Original Fire,271,2,23,None,218916,3577821,0.99
3,1,3394,3394,Broken City,271,2,23,None,228366,3728955,0.99
4,1,3395,3395,Somedays,271,2,23,None,213831,3497176,0.99
